In [35]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import Ollama
from langchain.chains import RetrievalQA
import json

# Paths to index and metadata files
index_file = r"D:\Portfolio Github\Airport_Chatbot\data\vector_index.faiss"
metadata_file = r"D:\Portfolio Github\Airport_Chatbot\data\metadata.json"

# Load the metadata
with open(metadata_file, 'r', encoding='utf-8') as f:
    metadata = json.load(f)

# Step 1: Load FAISS Index

# Initialize embeddings model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create FAISS index
vector_store = FAISS.from_texts(texts=index_file, embedding=embedding_model, metadatas=metadata[:len(index_file)])

# Save FAISS index
vector_store.save_local("./data/index_folder")

# Load FAISS index into LangChain's vector store
retriever = FAISS.load_local("./data/index_folder", embedding_model, allow_dangerous_deserialization=True)


# Step 2: Configure Ollama LLM
llm = Ollama(model="llama3.2", temperature = 0)  # Use the Ollama client with the chosen model

# Step 3: Build the RAG Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever.as_retriever(),
    return_source_documents=True
)

# Step 4: Query the RAG System
query = "What attractions are available at Jewel Changi Airport?"
result = qa_chain({"query":query})

# Display Results
print("Generated Answer:")
print(result['result'])
print("\nSources Used:")
for doc in result['source_documents']:
    print(f"- {doc.metadata['category']}: {doc.page_content}")




Generated Answer:
I don't have information on specific attractions available at Jewel Changi Airport. However, I can tell you that Jewel Changi Airport is a shopping and entertainment complex located within Changi Airport in Singapore, and it features various amenities such as restaurants, shops, and entertainment options like the Rain Vortex and Crystal Pavillion.

Sources Used:
- Attractions: l
- Attractions: b
- Attractions: b
- Attractions: e
